#### Imports

In [ ]:
import cv2
import numpy as np
import open3d as o3d
import os
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
import json
import shutil

from classes.CARLASemantics import SemanticColors

#### Directories

In [ ]:
BASE_DIR = "generated_data"
LIDAR_DIR = "LIDAR_TOP"
CAM_DIRS = ["CAM_FRONT", "CAM_FRONT_LEFT", "CAM_FRONT_RIGHT", "CAM_BACK", "CAM_BACK_LEFT", "CAM_BACK_RIGHT"]
SEMANTIC_CAM_DIRS =  ["SEMANTIC_CAM_FRONT", "SEMANTIC_CAM_FRONT_LEFT", "SEMANTIC_CAM_FRONT_RIGHT", "SEMANTIC_CAM_BACK", "SEMANTIC_CAM_BACK_LEFT", "SEMANTIC_CAM_BACK_RIGHT"]
DEPTH_CAM_DIRS = ["DEPTH_CAM_FRONT", "DEPTH_CAM_FRONT_LEFT", "DEPTH_CAM_FRONT_RIGHT", "DEPTH_CAM_BACK", "DEPTH_CAM_BACK_LEFT", "DEPTH_CAM_BACK_RIGHT"]
DEPTH_BEV_DIR = "DEPTH_BEV"
DEPTH_VISIBILITY_DIR = "DEPTH_VISIBILITY"

#### Intrinsics file (list of sensor configurations)

In [ ]:
INTRINSICS_FILEPATH = os.path.join("config", "nuscenes.intrinsics.json")
with open(INTRINSICS_FILEPATH, "r") as INTRINSICS_FILE:
    INTRINSICS = json.load(INTRINSICS_FILE)

print(INTRINSICS)

INTRINSICS_MATRICES = dict()
for SENSOR_NAME in INTRINSICS.keys():
    if SENSOR_NAME in CAM_DIRS:
        CAMERA_INTRINSICS = INTRINSICS[SENSOR_NAME]
        fx = CAMERA_INTRINSICS.get('fx', CAMERA_INTRINSICS.get('fl'))
        fy = CAMERA_INTRINSICS.get('fy', CAMERA_INTRINSICS.get('fl'))
        w, h = CAMERA_INTRINSICS.get('w'), CAMERA_INTRINSICS.get('h')
        ppx = w / 2
        ppy = h / 2
        d_type = CAMERA_INTRINSICS['disto_type']
        D = np.array(CAMERA_INTRINSICS['disto'])

        INTRINSICS_MATRICES[SENSOR_NAME] = np.array([[fx, 0, ppx, 0],
                                                     [0, fy, ppy, 0],
                                                     [0, 0, 1, 0]])


#### Post processing constants

In [ ]:
GRID_SIZE = 50
GRID_RESOLUTION = 0.5
MAX_SENSOR_SCAN_DISTANCE = 50

#### Filesystem methods

In [ ]:
def remove_files_in_dir(data_dir, string_to_find):
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if string_to_find in file:
                file_path = os.path.join(root, file)
                try:
                    os.remove(file_path)
                    print(f"Removed: {file_path}")
                except Exception as e:
                    print(f"Error removing {file_path}: {e}")

def get_all_filenames(dir, no_extension=False):
    if no_extension:
        return [filename.split(".")[0] for filename in os.listdir(dir)]
    return [filename for filename in os.listdir(dir)]

def clean_up_lidar_dir():
    LIDAR_DIR_PATH = os.path.join(BASE_DIR, LIDAR_DIR)
    remove_files_in_dir(LIDAR_DIR_PATH, ".bev.")

def clean_up_camera_dirs():
    for CAM_DIR in CAM_DIRS:
        CAM_DIR_PATH = os.path.join(BASE_DIR, CAM_DIR)
        remove_files_in_dir(CAM_DIR_PATH, ".pointcloud.")
        remove_files_in_dir(CAM_DIR_PATH, ".visibility.")
        remove_files_in_dir(CAM_DIR_PATH, ".fov.")

def clean_up_depth_camera_dirs():
    for DEPTH_CAM_DIR in DEPTH_CAM_DIRS:
        DEPTH_CAM_DIR_PATH = os.path.join(BASE_DIR, DEPTH_CAM_DIR)
        remove_files_in_dir(DEPTH_CAM_DIR_PATH, ".ply")
        remove_files_in_dir(DEPTH_CAM_DIR_PATH, ".fov.")
        remove_files_in_dir(DEPTH_CAM_DIR_PATH, ".visibility.")

def clean_up_depth_bev_dir():
    DEPTH_BEV_DIR_PATH = os.path.join(BASE_DIR, DEPTH_BEV_DIR)
    remove_files_in_dir(DEPTH_BEV_DIR_PATH, ".")
        
def save_point_cloud(file_path, point_cloud):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
    if type(point_cloud) is o3d.geometry.PointCloud:
        o3d.io.write_point_cloud(file_path, point_cloud)
        return
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    o3d.io.write_point_cloud(file_path, pcd)

def read_point_cloud(file_path):
    point_cloud = o3d.io.read_point_cloud(file_path)
    return np.asarray(point_cloud.points)

def save_image(file_path, mask):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
    cv2.imwrite(file_path, mask)

### Ground Truth from depth camera point clouds

#### Depth image parsing methods

In [ ]:
def calculate_depth_map_from_image(image):
    B = image[:, :, 0]
    G = image[:, :, 1]
    R = image[:, :, 2]
    normalized = (G + B * 256 + R * 256 * 256) / (256 * 256 - 1)
    depth_map = normalized * 1000
    return depth_map

def depth_image_to_point_cloud(depth_image, fov, max_distance=None):
    
    depth = calculate_depth_map_from_image(depth_image)
    if max_distance != None:
        depth[depth > max_distance] = 0.0 
    height, width = depth_image.shape[:2]

    # Create an intrinsic matrix from the camera parameters
    fx = fy = 0.5 * width / np.tan(0.5 * np.radians(fov))
    cx = width / 2.0
    cy = height / 2.0

    intrinsic = o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy)

    depth_o3d = o3d.geometry.Image(depth.astype(np.float32))
    
    pcd = o3d.geometry.PointCloud.create_from_depth_image(depth_o3d, intrinsic) #, depth_scale=1.0, depth_trunc=float(max_distance))
    points = np.asarray(pcd.points)
    swapped_points = points[:, [2, 0, 1]]
    swapped_points[:, 2] = swapped_points[:, 2] * (-1)
    pcd.points = o3d.utility.Vector3dVector(swapped_points)
    
    return pcd

def create_color_mask(image, colors, inverted=False):
    mask = np.full((image.shape[0], image.shape[1]), 0, dtype=np.uint8)
    
    B, G, R = image[:, :, 0], image[:, :, 1], image[:, :, 2]
    # Iterate through the list of colors
    for color in colors:
        # Extract color channels
        r, g, b = color
        # Create boolean masks for each channel comparison
        r_mask = R == r
        g_mask = G == g
        b_mask = B == b
        # Combine channel masks to get the final color mask
        color_mask = r_mask & g_mask & b_mask
        # Update the overall mask where any color matches
        mask[color_mask] = 255

    if inverted:
        mask = np.where(mask == 0, 255, 0).astype(np.uint8)
        return mask
    return mask

def mask_image(image, image_mask):
    masked = np.copy(image)
    masked[image_mask == 0] = 0
    return masked

#### Create FOV and visibility masks from depth images

In [ ]:
def generate_mask(grid, camera_matrix, T, image_size):
    homogeneous_grid = np.vstack([grid[i].flatten() for i in range(3)] + [np.ones(grid[0].size)])
    tfd_points = np.dot(T, homogeneous_grid)
    tfd_points = np.dot(camera_matrix, tfd_points)
    mask_x = tfd_points[0] < 0 # Exclude points behind the camera (x-axis is forward)
    tfd_points[0][tfd_points[0] == 0] = np.nan  # Replace zeros with NaN to avoid division by zero
    projected_points = tfd_points / tfd_points[0]
    mask = ((0 <= projected_points[1]) & (projected_points[1] < image_size[0]) &
            (0 <= projected_points[2]) & (projected_points[2] < image_size[1]) &
            mask_x)
    return mask.reshape(grid[0].shape)

def get_camera_fov_masks(camera_calibs, lidar_to_cam_tf_list=[], grid_size_m=50, resolution=0.5):
    whole_mask = np.zeros((int(grid_size_m / resolution), int(grid_size_m / resolution)))
    x, y = np.meshgrid(np.arange(whole_mask.shape[1]), np.arange(whole_mask.shape[0]))
    x = x - whole_mask.shape[1] / 2
    y = y - whole_mask.shape[0] / 2
    z = np.zeros_like(x)
    bev_grid = np.array([x, y, z])
    bev_grid = np.expand_dims(bev_grid, axis=-1)

    cam_masks = {}

    for i, (camera, calib) in enumerate(camera_calibs.items()):
        intrinsic = calib['K']

        w, h = calib['w'], calib['h']

        if lidar_to_cam_tf_list:
            cam_T_lidar = np.linalg.inv(lidar_to_cam_tf_list[i])
            
        else:
            cam_T_lidar = np.linalg.inv(calib['T'])

        mask = generate_mask(bev_grid, camera_matrix=intrinsic, T=cam_T_lidar, image_size=(w, h))
        visible_bev = np.array([dim[mask] for dim in bev_grid])

        mask_ref = visible_bev[:2]
        cam_mask = np.zeros_like(mask)

        mask_ref[1] += mask.shape[1] / 2
        mask_ref[0] += mask.shape[0] / 2
        mask_ref = np.round(mask_ref).astype(int)

        cam_mask[mask_ref[1], mask_ref[0]] = 1
        cam_mask = cam_mask.squeeze(-1)

        cam_masks[camera] = cam_mask
    return cam_masks

def get_fov_mask(image, transformation_matrix, camera_intrinsics_matrix):
    # Example camera calibration data for two cameras
    camera_calibs = {
        'camera': {
            'K': camera_intrinsics_matrix,
            'w': image.shape[1],  # Image width
            'h': image.shape[0],  # Image height
            'T': transformation_matrix
        }
    }
    # Generate the camera FOV masks
    cam_masks = get_camera_fov_masks(camera_calibs, grid_size_m=GRID_SIZE, resolution=GRID_RESOLUTION)
    fov_mask = np.asarray(cam_masks["camera"], dtype=np.uint8) * 255
    
    return fov_mask

def extract_XY_translation_from_transform(transform_matrix):
    return transform_matrix[:, 3][:2]

def extract_yaw_from_transform(transform_matrix):
    yaw_radians =  np.arctan2(transform_matrix[1][0], transform_matrix[0][0])
    return np.degrees(yaw_radians)

def calculate_grid_coords(coords, grid_size, resolution):
    grid_center = int(grid_size / resolution / 2), int(grid_size / resolution / 2)
    grid_coords = int(grid_center[0] - coords[0] / resolution), int(grid_center[1] - coords[1] / resolution)
    return grid_coords

def calculate_relative_forward_direction(angle1, angle2):
    angle = (angle1 - angle2) + 90
    if angle < 0:
        angle += 360
    if angle > 360:
        angle -= 360
    return angle

def is_angle_in_range_radians(angle, start_angle, end_angle):
        if (angle < 0):
            angle += 2 * np.pi

        angle = angle % (2 * np.pi)
        start_angle = start_angle % (2 * np.pi)
        end_angle = end_angle % (2 * np.pi)
        
        if start_angle < end_angle:
            # Normal case where the range does not cross 0 radians
            return start_angle <= angle <= end_angle
        else:
            # Case where the range crosses 0 radians
            return angle >= start_angle or angle <= end_angle

def create_fov_mask(grid_size, grid_resolution, sensor_coords, sensor_direction, fov_degrees):    
    # Create an empty mask
    mask = np.zeros((int(grid_size / grid_resolution), int(grid_size / grid_resolution)), dtype=np.uint8)
    
    # Convert the FOV and direction to radians
    sensor_direction_rad = np.deg2rad(sensor_direction)
    fov_rad = np.deg2rad(fov_degrees)
    
    # Calculate the angle range for the FOV
    fov_start = sensor_direction_rad - fov_rad / 2
    fov_end = sensor_direction_rad + fov_rad / 2
    
    # Calculate the position of the sensor
    y0, x0 = sensor_coords
    
    # Iterate over each point in the grid
    for y in range(mask.shape[0]):
        for x in range(mask.shape[1]):
            # Calculate the angle from the sensor to this point
            dy = y - y0
            dx = x - x0

            angle = np.arctan2(dy, dx)

            # Check if the angle is within the FOV
            if is_angle_in_range_radians(angle, fov_start, fov_end):
                mask[y, x] = 255
    
    
    return mask

clean_up_depth_camera_dirs()

for (SEM_DIR, DEPTH_DIR, CAM_DIR) in zip(SEMANTIC_CAM_DIRS, DEPTH_CAM_DIRS, CAM_DIRS):
    for filename in os.listdir(os.path.join(BASE_DIR, DEPTH_DIR)):
        if filename.endswith('.png'):
            depth_image_path = os.path.join(BASE_DIR, DEPTH_DIR, filename)
            depth_image = cv2.imread(depth_image_path)
            depth_camera_fov = INTRINSICS.get(CAM_DIR).get("fov")
            depth_image_point_cloud = depth_image_to_point_cloud(depth_image, fov=depth_camera_fov, max_distance=MAX_SENSOR_SCAN_DISTANCE)
            
            depth_camera_transform_path = os.path.join(BASE_DIR, DEPTH_DIR, filename.replace(".png", ".npy"))
            depth_camera_transform = np.load(depth_camera_transform_path)
            lidar_transform = np.load(os.path.join(BASE_DIR, LIDAR_DIR, filename.replace(".png", ".npy")))
            lidar_transform_inv = np.linalg.inv(lidar_transform)
            combined_transform = np.dot(lidar_transform_inv, depth_camera_transform)
            depth_image_point_cloud.transform(combined_transform)

            depth_image_point_cloud_path = os.path.join(BASE_DIR, DEPTH_DIR, filename.replace(".png", ".ply"))
            print(f"Added: {depth_image_point_cloud_path}")
            save_point_cloud(depth_image_point_cloud_path, depth_image_point_cloud)
            
            
            """
            fov_mask_image = get_fov_mask(
                depth_image,
                combined_transform,
                INTRINSICS_MATRICES[CAM_DIR] 
            )
            """
            lidar_coords = extract_XY_translation_from_transform(lidar_transform)
            lidar_forward_direction = extract_yaw_from_transform(lidar_transform)
            camera_coords = extract_XY_translation_from_transform(depth_camera_transform)
            camera_forward_direction = extract_yaw_from_transform(depth_camera_transform)
            relative_coords = lidar_coords - camera_coords
            camera_grid_coords = calculate_grid_coords(relative_coords, GRID_SIZE, GRID_RESOLUTION)
            camera_grid_forward_direction = calculate_relative_forward_direction(lidar_forward_direction, camera_forward_direction)
            camera_fov = INTRINSICS.get(CAM_DIR).get("fov")
            fov_mask = create_fov_mask(GRID_SIZE, GRID_RESOLUTION, camera_grid_coords, camera_grid_forward_direction, camera_fov)
            fov_mask_path = os.path.join(BASE_DIR, DEPTH_DIR, filename.replace(".png", ".fov.png"))
            print(f"Added: {fov_mask_path}")
            save_image(fov_mask_path, fov_mask)
            
            def rasterize_to_bev(points, resolution=0.5, grid_size=25):
                bev_map = np.zeros((int(grid_size / resolution), int(grid_size / resolution)))
                # Converting to grid coordinates
                grid_coords = np.floor(points[:, :2] / resolution).astype(np.int32) + int(grid_size // (2 * resolution))
                
                # Ensure that grid coordinates are within the bounds of the BEV map
                valid_points = (grid_coords[:, 0] >= 0) & (grid_coords[:, 0] < bev_map.shape[0]) & \
                            (grid_coords[:, 1] >= 0) & (grid_coords[:, 1] < bev_map.shape[1])
                # Populate the BEV map with occupancy
                bev_map[grid_coords[valid_points, 0], grid_coords[valid_points, 1]] = 255
                return bev_map

            visibility_mask = rasterize_to_bev(np.asarray(depth_image_point_cloud.points), resolution=GRID_RESOLUTION, grid_size=GRID_SIZE)
            visibility_mask_path = os.path.join(BASE_DIR, DEPTH_DIR, filename.replace(".png", ".visibility.png"))
            print(f"Added: {visibility_mask_path}")
            save_image(visibility_mask_path, visibility_mask)

timestamps = get_all_filenames(os.path.join(BASE_DIR, DEPTH_CAM_DIRS[0]), no_extension=True)
for timestamp in timestamps:
    cumulative_visibility_mask = np.zeros((int(GRID_SIZE / GRID_RESOLUTION), int(GRID_SIZE / GRID_RESOLUTION)))
    for (SEM_DIR, DEPTH_DIR, CAM_DIR) in zip(SEMANTIC_CAM_DIRS, DEPTH_CAM_DIRS, CAM_DIRS):
        depth_image_path = os.path.join(BASE_DIR, DEPTH_DIR, f"{timestamp}.visibility.png")
        visibility_mask = cv2.imread(depth_image_path, cv2.IMREAD_GRAYSCALE)
        cumulative_visibility_mask[visibility_mask > 0] = 255
    cumulative_visibility_mask_path = os.path.join(BASE_DIR, DEPTH_VISIBILITY_DIR, f"{timestamp}.bev.png")
    save_image(cumulative_visibility_mask_path, cumulative_visibility_mask)
    print(f"Added: {cumulative_visibility_mask_path}")

    lidar_transform_path = os.path.join(BASE_DIR, LIDAR_DIR, f"{timestamp}.npy")
    lidar_transform = np.load(lidar_transform_path)
    transformation_file_path = os.path.join(BASE_DIR, DEPTH_VISIBILITY_DIR, f"{timestamp}.npy")
    np.save(transformation_file_path, lidar_transform)
    print(f"Added: {transformation_file_path}")


#### Create point clouds and occupancy maps from depth images

In [ ]:
def get_all_filenames(dir, no_extension=False):
    if no_extension:
        return [filename.split(".")[0] for filename in os.listdir(dir)]
    return [filename for filename in os.listdir(dir)]

def get_obstacle_point_cloud(depth_image, semantic_image, fov):
    obstacles_mask = create_color_mask(
        semantic_image, 
        colors=[
            SemanticColors.ROADLINE.value, SemanticColors.ROAD.value, SemanticColors.SIDEWALK.value,
            SemanticColors.GROUND.value, SemanticColors.WATER.value, SemanticColors.TERRAIN.value,
            SemanticColors.SKY.value
        ],
        inverted=True
    )
    obstacles_depth_image = mask_image(depth_image, obstacles_mask)

    ground_mask = create_color_mask(semantic_image, colors=[
        SemanticColors.ROADLINE.value, SemanticColors.ROAD.value, SemanticColors.SIDEWALK.value,
        SemanticColors.GROUND.value, SemanticColors.WATER.value, SemanticColors.TERRAIN.value
    ])
    ground_depth_image = mask_image(depth_image, ground_mask)

    obstacles_point_cloud = depth_image_to_point_cloud(obstacles_depth_image, fov=fov, max_distance=MAX_SENSOR_SCAN_DISTANCE)
    ground_point_cloud = depth_image_to_point_cloud(ground_depth_image, fov=fov, max_distance=MAX_SENSOR_SCAN_DISTANCE)

    def gen_mesh(pcd): 
        try:
            points = np.asarray(pcd.points)
        except:
            points = pcd
        tri = Delaunay(points[:, :2])  # We only use the X and Y coordinates
        mesh = o3d.geometry.TriangleMesh()
        mesh.vertices = o3d.utility.Vector3dVector(points)
        mesh.triangles = o3d.utility.Vector3iVector(tri.simplices)
        return mesh
    
    def mesh_to_cloud_signed_distances(o3d_mesh: o3d.t.geometry.TriangleMesh, cloud: o3d.t.geometry.PointCloud) -> np.ndarray:
        scene = o3d.t.geometry.RaycastingScene()
        _ = scene.add_triangles(o3d_mesh)
        sdf = scene.compute_signed_distance(cloud.point.positions)
        return sdf.numpy()

    def filter_points_far_from_mesh(pcd, distances, t1, t2):
        indices1 = np.where((distances > t1) & (distances <= t2))[0]
        indices2 = np.where(distances < t1)[0]
        objects = pcd.select_by_index(indices1)
        ground = pcd.select_by_index(indices2)
        return objects, ground

    def remove_points_far_from_mesh(pcd, mesh, height_range=(0.4, 2)):
        mesh_t = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
        tpcd = o3d.t.geometry.PointCloud.from_legacy(pcd)
        sdf = mesh_to_cloud_signed_distances(mesh_t, tpcd)
        sdf = np.abs(sdf)
        obstacles, ground = filter_points_far_from_mesh(pcd, sdf, *height_range)
        return obstacles, ground
    try:
        ground_mesh = gen_mesh(ground_point_cloud)
        obstacles_point_cloud, ground_point_cloud = remove_points_far_from_mesh(obstacles_point_cloud, ground_mesh, height_range=(0.2, 3))
    except:
        pass
    return obstacles_point_cloud

clean_up_depth_bev_dir()

timestamps = get_all_filenames(os.path.join(BASE_DIR, DEPTH_CAM_DIRS[0]), no_extension=True)
for timestamp in timestamps:
    cumulative_point_cloud = o3d.geometry.PointCloud()
    for (SEM_DIR, DEPTH_DIR, CAM_DIR) in zip(SEMANTIC_CAM_DIRS, DEPTH_CAM_DIRS, CAM_DIRS):
        depth_image_path = os.path.join(BASE_DIR, DEPTH_DIR, f"{timestamp}.png")
        depth_image = cv2.imread(depth_image_path)
        depth_camera_fov = INTRINSICS.get(CAM_DIR).get("fov")
        semantic_image_path = os.path.join(BASE_DIR, SEM_DIR, f"{timestamp}.png")
        semantic_image = cv2.imread(semantic_image_path)
        depth_point_cloud = get_obstacle_point_cloud(depth_image, semantic_image, depth_camera_fov)
        depth_camera_transform_path = os.path.join(BASE_DIR, DEPTH_DIR, f"{timestamp}.npy")
        depth_camera_transform = np.load(depth_camera_transform_path)
        depth_point_cloud.transform(depth_camera_transform)
        cumulative_point_cloud.points.extend(depth_point_cloud.points)
    reference_transform_matrix = np.load(os.path.join(BASE_DIR, LIDAR_DIR, f"{timestamp}.npy"))
    cumulative_point_cloud.transform(np.linalg.inv(reference_transform_matrix))
    cumulative_point_cloud_file_path = os.path.join(BASE_DIR, DEPTH_BEV_DIR, f"{timestamp}.ply")
    save_point_cloud(cumulative_point_cloud_file_path, cumulative_point_cloud)
    print(f"Added: {cumulative_point_cloud_file_path}")

    transformation_file_path = os.path.join(BASE_DIR, DEPTH_BEV_DIR, f"{timestamp}.npy")
    np.save(transformation_file_path, reference_transform_matrix)
    print(f"Added: {transformation_file_path}")

    occupancy_image = rasterize_to_bev(np.asarray(cumulative_point_cloud.points), resolution=GRID_RESOLUTION, grid_size=GRID_SIZE)
    occupancy_image_path = os.path.join(BASE_DIR, DEPTH_BEV_DIR, f"{timestamp}.bev.png")
    save_image(occupancy_image_path, occupancy_image)
    print(f"Added: {occupancy_image_path}")

### Ground Truth from semantic LIDAR point clouds

#### Create occupancy maps from LIDAR point clouds & FOV and visibility masks for camera sensors

In [ ]:
def rasterize_to_bev(points, resolution=0.5, grid_size=25):
    bev_map = np.zeros((int(grid_size / resolution), int(grid_size / resolution)))
    # Converting to grid coordinates
    grid_coords = np.floor(points[:, :2] / resolution).astype(np.int32) + int(grid_size // (2 * resolution))
    
    # Ensure that grid coordinates are within the bounds of the BEV map
    valid_points = (grid_coords[:, 0] >= 0) & (grid_coords[:, 0] < bev_map.shape[0]) & \
                   (grid_coords[:, 1] >= 0) & (grid_coords[:, 1] < bev_map.shape[1])
    # Populate the BEV map with occupancy
    bev_map[grid_coords[valid_points, 0], grid_coords[valid_points, 1]] = 255
    return bev_map

clean_up_lidar_dir()

for filename in os.listdir(os.path.join(BASE_DIR, LIDAR_DIR)):
    if filename.endswith('.ply'):
        point_cloud_file_path = os.path.join(BASE_DIR, LIDAR_DIR, filename)
        point_cloud = read_point_cloud(point_cloud_file_path)
        bev_rasterized_image = rasterize_to_bev(point_cloud, resolution=GRID_RESOLUTION, grid_size=GRID_SIZE)
        bev_file_path = os.path.join(BASE_DIR, LIDAR_DIR, filename.replace(".ply", ".bev.mask.png"))
        print(f"Added: {bev_file_path}")
        save_image(bev_file_path, bev_rasterized_image)

def extract_XY_translation_from_transform(transform_matrix):
    return transform_matrix[:, 3][:2]

def extract_yaw_from_transform(transform_matrix):
    yaw_radians =  np.arctan2(transform_matrix[1][0], transform_matrix[0][0])
    return np.degrees(yaw_radians)

def calculate_grid_coords(coords, grid_size, resolution=0.5):
    grid_center = (grid_size // 2, grid_size // 2)
    return (int(grid_center[0] - coords[0]), int(grid_center[1] - coords[1]))

def calculate_relative_forward_direction(angle1, angle2):
    angle = (angle1 - angle2) + 90
    if angle < 0:
        angle += 360
    if angle > 360:
        angle -= 360
    return angle

def is_angle_in_range_radians(angle, start_angle, end_angle):
        angle = angle % (2 * np.pi)
        start_angle = start_angle % (2 * np.pi)
        end_angle = end_angle % (2 * np.pi)
        
        if start_angle < end_angle:
            # Normal case where the range does not cross 0 radians
            return start_angle <= angle <= end_angle
        else:
            # Case where the range crosses 0 radians
            return angle >= start_angle or angle <= end_angle

def create_fov_mask(grid_size, sensor_coords, sensor_direction, fov_degrees):    
    # Create an empty mask
    mask = np.zeros((grid_size, grid_size), dtype=np.uint8)
    
    # Convert the FOV and direction to radians
    sensor_direction_rad = np.deg2rad(sensor_direction)
    fov_rad = np.deg2rad(fov_degrees)
    
    # Calculate the angle range for the FOV
    fov_start = sensor_direction_rad - fov_rad / 2
    fov_end = sensor_direction_rad + fov_rad / 2
    
    # Calculate the position of the sensor
    y0, x0 = sensor_coords
    
    # Iterate over each point in the grid
    for y in range(grid_size):
        for x in range(grid_size):
            # Calculate the angle from the sensor to this point
            dy = y - y0
            dx = x - x0

            angle = np.arctan2(dy, dx)
            if (angle < 0):
                angle += 2 * np.pi

            # Check if the angle is within the FOV
            if is_angle_in_range_radians(angle, fov_start, fov_end):
                mask[y, x] = 255
    
    return mask

def create_visibility_mask(world_grid, fov_grid, sensor_coords):
    
    def find_path(start_point, end_point):
        # https://en.wikipedia.org/wiki/Bresenham%27s_line_algorithm
        path = []
        
        x0, y0 = start_point
        x1, y1 = end_point
        dx = abs(x1 - x0)
        dy = abs(y1 - y0)
        sx = 1 if x0 < x1 else -1
        sy = 1 if y0 < y1 else -1
        err = dx - dy
        
        while True:
            path.append((x0, y0))
            if x0 == x1 and y0 == y1:
                break
            e2 = 2 * err
            if e2 > -dy:
                err -= dy
                x0 += sx
            if e2 < dx:
                err += dx
                y0 += sy
        
        return path

    def remove_obstructed_sections(world, path, visibility_mask):
        obstruction_detected = False
        for x, y in path:
            if world[y][x] == 255:
                obstruction_detected = True
            if obstruction_detected:
                visibility_mask[y][x] = 0

    def get_fov_coordinates(array):
        return np.argwhere(array == 255)

    visibility_mask = np.copy(fov_grid)
    all_fov_coords = get_fov_coordinates(fov_grid)
    
    for (y, x) in all_fov_coords:
        path = find_path(sensor_coords, (x, y))
        remove_obstructed_sections(world_grid, path, visibility_mask)
    return visibility_mask


clean_up_camera_dirs()

for filename in os.listdir(os.path.join(BASE_DIR, LIDAR_DIR)): 
    if filename.endswith('.npy'):
        lidar_npy_file = os.path.join(BASE_DIR, LIDAR_DIR, filename)
        lidar_transform_matrix = np.load(lidar_npy_file)
        lidar_coords = extract_XY_translation_from_transform(lidar_transform_matrix)
        lidar_forward_direction = extract_yaw_from_transform(lidar_transform_matrix)
        lidar_bev_file = os.path.join(BASE_DIR, LIDAR_DIR, filename.replace('.npy', '.bev.mask.png'))
        lidar_bev_mask = cv2.imread(lidar_bev_file, cv2.IMREAD_GRAYSCALE)

        for CAM_DIR in CAM_DIRS:
            camera_npy_file = os.path.join(BASE_DIR, CAM_DIR, filename)
            camera_transform_matrix = np.load(camera_npy_file)
            camera_coords = extract_XY_translation_from_transform(camera_transform_matrix)
            camera_forward_direction = extract_yaw_from_transform(camera_transform_matrix)  # degrees
            
            relative_coords = lidar_coords - camera_coords
            camera_grid_coords = calculate_grid_coords(relative_coords, GRID_SIZE)
            camera_grid_forward_direction = calculate_relative_forward_direction(lidar_forward_direction, camera_forward_direction)

            camera_fov = INTRINSICS.get(CAM_DIR).get("fov")
            fov_mask = create_fov_mask(GRID_SIZE, camera_grid_coords, camera_grid_forward_direction, camera_fov)
            fov_mask_file_path = os.path.join(BASE_DIR, CAM_DIR, filename.replace('.npy', '.fov.mask.png'))
            print(f"Added: {fov_mask_file_path}")
            save_image(fov_mask_file_path, fov_mask)

            visibility_mask = create_visibility_mask(lidar_bev_mask, fov_mask, camera_grid_coords)
            visibility_mask_file_path = os.path.join(BASE_DIR, CAM_DIR, filename.replace('.npy', '.visibility.mask.png'))
            print(f"Added: {visibility_mask_file_path}")
            save_image(visibility_mask_file_path, visibility_mask)   

### Export processed files to target directory (in suitable directory tree format for machine learning pipeline) 

In [ ]:
TARGET_DIR = "processed_data"
TARGET_CAM_DIRS = [
    "AGV_sensors_cameras_front_raw", "AGV_sensors_cameras_front_left_raw", "AGV_sensors_cameras_front_right_raw", 
    "AGV_sensors_cameras_back_raw", "AGV_sensors_cameras_back_left_raw", "AGV_sensors_cameras_back_right_raw"
]
TARGET_LIDAR_DIR = "AGV_sensors_lidar_top_raw"
TARGET_FOV_MASKS_DIR = "fov_masks"
TARGET_BEVS_DIR = "sdf"
TARGET_VISIBILITY_MASKS_DIR = "visibility_masks"
TARGET_CUMULATIVE_MASKS_DIR = "cumulative_masks"

In [ ]:
def clean_up_target_dir():
    if not os.path.exists(TARGET_DIR):
        return
    for subdir in os.listdir(TARGET_DIR):
        shutil.rmtree(os.path.join(TARGET_DIR, subdir))

In [ ]:
def copy_files_with_extensions(source_dir, target_dir, extensions, exclude_strings=[]):
    # Ensure the target directory exists
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    
    # Iterate through the files in the source directory
    for root, _, files in os.walk(source_dir):
        for file in files:
            # Check if the file has one of the specified extensions
            if any(file.endswith(ext) for ext in extensions):
                # Check if the file name contains any of the exclude strings
                if any(exclude_str in file for exclude_str in exclude_strings):
                    print(f"Excluded {file}")
                    continue
                
                # Construct full file paths
                source_file = os.path.join(root, file)
                target_file = os.path.join(target_dir, file)
                
                # Copy the file to the target directory
                shutil.copy2(source_file, target_file)
                print(f"Copied {source_file} to {target_file}")

clean_up_target_dir()

# Export RGB camera data
source_dirs = [os.path.join(BASE_DIR, CAM_DIR) for CAM_DIR in CAM_DIRS]
target_dirs = [os.path.join(TARGET_DIR, CAM_DIR) for CAM_DIR in TARGET_CAM_DIRS]
for source_dir, target_dir in zip(source_dirs, target_dirs):
    extensions = [".png", ".npy"]
    exclude_strings = ["fov", "visibility"]
    copy_files_with_extensions(source_dir, target_dir, extensions, exclude_strings=exclude_strings)

# Export lidar data
source_dir = os.path.join(BASE_DIR, LIDAR_DIR)
target_dir = os.path.join(TARGET_DIR, TARGET_LIDAR_DIR)
extensions = [".ply", ".npy"]
copy_files_with_extensions(source_dir, target_dir, extensions)

# Export FOV masks
source_dirs = [os.path.join(BASE_DIR, DEPTH_CAM_DIR) for DEPTH_CAM_DIR in DEPTH_CAM_DIRS]
target_dirs = [os.path.join(TARGET_DIR, TARGET_FOV_MASKS_DIR, CAM_DIR) for CAM_DIR in TARGET_CAM_DIRS]
for source_dir, target_dir in zip(source_dirs, target_dirs):
    extensions = [".fov.png", ".npy"]
    copy_files_with_extensions(source_dir, target_dir, extensions)

# Export visibility masks
source_dirs = [os.path.join(BASE_DIR, DEPTH_CAM_DIR) for DEPTH_CAM_DIR in DEPTH_CAM_DIRS]
target_dirs = [os.path.join(TARGET_DIR, TARGET_VISIBILITY_MASKS_DIR, CAM_DIR) for CAM_DIR in TARGET_CAM_DIRS]
for source_dir, target_dir in zip(source_dirs, target_dirs):
    extensions = [".visibility.png", ".npy"]
    copy_files_with_extensions(source_dir, target_dir, extensions)

# Export cumulative masks
source_dir = os.path.join(BASE_DIR, DEPTH_VISIBILITY_DIR)
target_dir = os.path.join(TARGET_DIR, TARGET_VISIBILITY_MASKS_DIR, TARGET_CUMULATIVE_MASKS_DIR)
extensions = [".bev.png", ".npy"]
copy_files_with_extensions(source_dir, target_dir, extensions)

# Export BEV masks
source_dir = os.path.join(BASE_DIR, DEPTH_BEV_DIR)
target_dir = os.path.join(TARGET_DIR, TARGET_BEVS_DIR)
extensions = [".png", ".npy"]
copy_files_with_extensions(source_dir, target_dir, extensions)
